#### Задача 1

Известно, что генеральная совокупность распределена нормально с известным средним квадратическим отклонением 16. Найти доверительный интервал для оценки математического ожидания mu с надёжностью 0.95, если выборочное среднее равно 80, а объём равен 256.

In [5]:
import numpy as np
from scipy import stats

Так как известно,что распределение нормальное и известно среднее квадратическое отклонение, то подходит Z-статистика.
$$Z = \dfrac{\overline{X} - \mu}{\sigma / \sqrt{n}},$$

Доверительный интервал
<center>
    <div style="width: 350px;border:1px solid black;">
        $P \left( \overline{X} + t_{\alpha / 2} \cdot \dfrac{\sigma}{\sqrt{n}} \leq \mu \leq \overline{X} + t_{1 - \alpha / 2} \cdot \dfrac{\sigma}{\sqrt{n}} \right) = p$
    </div>
</center>

In [12]:
mean = 80
n = 256
std = 16
p = 0.95
alpha = 1 - p
alpha = 0.05

In [13]:
#Квантили
t1 = stats.norm.ppf(alpha / 2)
t2 = stats.norm.ppf(1 - alpha / 2)
t1,t2

(-1.9599639845400545, 1.959963984540054)

In [14]:
#Доверительный интервал
(mean + t1*std/np.sqrt(n),mean + t2*std/np.sqrt(n))

(78.04003601545995, 81.95996398454005)

#### Задача 2

Продавец утверждает, что средний вес пачки печенья составляет 200 г. Из партии извлечена выборка из 10 пачек. Вес каждой пачки составляет:

202, 203, 199, 197, 195, 201, 200, 204, 194, 190

Известно, что их веса распределены нормально.
1. Верно ли утверждение продавца, если учитывать, что уровень значимости равен 1%?
2. Найдите P-значение для данного теста.

In [18]:
samples = np.array([202, 203, 199, 197, 195, 201, 200, 204, 194, 190])

Известно,что $\mu$ = 200, распределение нормальное и среднее квадратическое отклонение неизвестно, то подходит t-статистика. 
$$t = \dfrac{\overline{X} - \mu}{\sigma_X / \sqrt{n}},$$
В нашем случае это:
$$t = \dfrac{198.5 - 200}{4.45\cdot\sqrt{10}}$$

In [24]:
def statistic(samples: np.ndarray) -> float:
    return (samples.mean() - 200) / (samples.std(ddof=1) / np.sqrt(samples.shape[0]))

In [30]:
std = samples.std(ddof=1)
n = samples.shape[0]
mean = samples.mean()
n, mean, std

(10, 198.5, 4.453463071962462)

In [34]:
# Значение статистики
t = statistic(samples)
t

-1.0651074037450896

1.Верно ли утверждение продавца, если учитывать, что уровень значимости равен 1%?

Уровень значимости $\alpha = 0.01$

Распределение t-Стъюдента с параметром df = n -1

In [33]:
#Доверительный интервал
alpha = 0.01
t1 = stats.t.ppf(alpha / 2, df=n - 1)
t2 = stats.t.ppf(1 - alpha / 2, df=n - 1)

t1, t2

(-3.2498355440153697, 3.2498355440153697)

In [39]:
print(alpha, t1 <= t <= t2) # Значение статистики находится в пределах доверительного интервала. 
# Значит, утверждение продавца верно. Нулевая гипотеза верна.

0.01 True


2.Найдите P-значение для данного теста.

В нашем случае критическая область двусторонняя.
Для двухсторонней области $\Omega_\alpha = \left( -\infty, t_{\alpha / 2} \right) \cup \left( t_{1 - \alpha / 2} , \infty \right)$ нужна комбинация двух: $$P = 2 \cdot \min (P_l, P_r)$$

Для правосторонней области $\Omega_\alpha = \left( t_{1 - \alpha}, \infty \right)$ имеем условие $t_{1 - \alpha} = t$, откуда $$P_r = 1 - F_t(t)$$

Для левосторонней области $\Omega_\alpha = \left( -\infty, t_\alpha \right)$, условие $t_\alpha = t$, откуда $$P_l = F_t(t)$$

In [43]:
p_left = stats.t.cdf(t, df=n - 1)
p_right = 1 - stats.t.cdf(t, df=n - 1)

pvalue = 2 * min(p_left, p_right)

pvalue #p-значение для теста

0.3145666035247285

In [44]:
print(alpha, pvalue, alpha < pvalue) # Значит нулевую гипотезу можно принять

0.01 0.3145666035247285 True


#### Задача 3

Дана выборка диаметров подшипников из примера 1 с занятия 5:

samples = [0.6603, 0.9466, 0.5968, 1.3792, 1.5481, 0.7515, 1.0681, 1.1134,
1.2088, 1.701 , 1.0282, 1.3579, 1.0191, 1.1784, 1.1168, 1.1372,
0.7273, 1.3958, 0.8665, 1.5112, 1.161 , 1.0232, 1.0865, 1.02 ]

Предполагая, что диаметры подшипников распределены нормально, проверьте гипотезу о том, что дисперсия случайной величины равна 0.0625 при уровне значимости alpha = 0.05. Что для этого нужно знать:
1. Альтернативная гипотеза двухсторонняя.
2. Статистика для теста: H = (n - 1) * sample_variance / variance, где n - число элементов в выборке, sample_variance - несмещённая оценка дисперсии, variance - утверждаемая нулевой гипотезой дисперсия.
3. Эта статистика в предположении верности нулевой гипотезы имеет распределение хи-квадрат с параметром df = n - 1. Её квантили можно найти с помощью функции scipy.stats.chi2.ppf.

In [46]:
samples_b = np.array([0.6603, 0.9466, 0.5968, 1.3792, 1.5481, 0.7515, 1.0681, 1.1134,
                      1.2088, 1.701 , 1.0282, 1.3579, 1.0191, 1.1784, 1.1168, 1.1372,
                      0.7273, 1.3958, 0.8665, 1.5112, 1.161 , 1.0232, 1.0865, 1.02 ])

$H_0$ заключается в том, что $D(X) = \sigma^2 = 0.0625$ $\alpha = 0.05$

Статистика для теста $H = (n-1)\cdot \dfrac{\sigma_X^2}{\sigma^2}$

In [56]:
var = samples_b.var(ddof=1)
n = samples_b.shape[0]
d = 0.0625
alpha = 0.05
n, var, d, alpha

(24, 0.07804495041666669, 0.0625, 0.05)

In [57]:
def statistic_H(samples: np.ndarray) -> float:
    return (samples.shape[0] - 1) * (samples.var(ddof=1) /d)

In [58]:
H = statistic_H(samples_b)
H

28.720541753333343

Эта статистика в предположении верности нулевой гипотезы имеет распределение хи-квадрат с параметром df = n - 1. Её квантили можно найти с помощью функции scipy.stats.chi2.ppf.

In [59]:
#Доверительный интервал
t1 = stats.chi2.ppf(alpha / 2, df=n - 1)
t2 = stats.chi2.ppf(1 - alpha / 2, df=n - 1)

t1, t2

(11.688551922452438, 38.0756272503558)

In [61]:
print(alpha, d, t1 <= H <= t2) 
#Значение статистики находится внутри доверительного интервала, не попало в критическую область. 
#Значит, нулевую гипотезу можно принять.

0.05 0.0625 True
